Разобраться с моделькой перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)



In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
os.getcwd()

'G:\\Мой диск\\Colab Notebooks\\NLP\\Lesson10'

In [3]:
path_to_file = "rus.txt"

In [4]:
def preprocess_sentence(w):
    w = w.lower().strip()
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
    w = w.strip()
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]
    return zip(*word_pairs)

In [6]:
en, ru = create_dataset(path_to_file, None)

In [7]:
en

('<start> go . <end>',
 '<start> go . <end>',
 '<start> go . <end>',
 '<start> hi . <end>',
 '<start> hi . <end>',
 '<start> hi . <end>',
 '<start> hi . <end>',
 '<start> hi . <end>',
 '<start> hi . <end>',
 '<start> run ! <end>',
 '<start> run ! <end>',
 '<start> run . <end>',
 '<start> run . <end>',
 '<start> who ? <end>',
 '<start> wow ! <end>',
 '<start> wow ! <end>',
 '<start> wow ! <end>',
 '<start> wow ! <end>',
 '<start> wow ! <end>',
 '<start> wow ! <end>',
 '<start> duck ! <end>',
 '<start> fire ! <end>',
 '<start> fire ! <end>',
 '<start> help ! <end>',
 '<start> help ! <end>',
 '<start> help ! <end>',
 '<start> hide . <end>',
 '<start> hide . <end>',
 '<start> hide . <end>',
 '<start> hide . <end>',
 '<start> jump ! <end>',
 '<start> jump ! <end>',
 '<start> jump . <end>',
 '<start> jump . <end>',
 '<start> stay . <end>',
 '<start> stay . <end>',
 '<start> stay . <end>',
 '<start> stay . <end>',
 '<start> stop ! <end>',
 '<start> stop ! <end>',
 '<start> stop ! <end>',
 '<s

In [8]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [9]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [10]:
len(en), len(ru)

(451436, 451436)

In [11]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [12]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [17]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [18]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
13 ----> мы
88 ----> были
616 ----> счастливы
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
17 ----> we
79 ----> were
131 ----> happy
3 ----> .
2 ----> <end>


In [19]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [20]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [22]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [23]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [24]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)



In [25]:
decoder_sample_x.shape

TensorShape([64, 7335])

In [26]:
decoder_sample_h.shape

TensorShape([64, 1024])

In [27]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [28]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [29]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)

            loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [30]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.7285
Epoch 1 Batch 100 Loss 2.0302
Epoch 1 Batch 200 Loss 1.8823
Epoch 1 Batch 300 Loss 1.6206
Epoch 1 Batch 400 Loss 1.5212
Epoch 1 Batch 500 Loss 1.3477
Epoch 1 Batch 600 Loss 1.2831
Epoch 1 Batch 700 Loss 1.2307
Epoch 1 Batch 800 Loss 1.2361
Epoch 1 Batch 900 Loss 1.1497
Epoch 1 Batch 1000 Loss 1.1108
Epoch 1 Batch 1100 Loss 1.0012
Epoch 1 Batch 1200 Loss 1.1429
Epoch 1 Loss 1.4017
Time taken for 1 epoch 1307.419626712799 sec

Epoch 2 Batch 0 Loss 0.8550
Epoch 2 Batch 100 Loss 0.8577
Epoch 2 Batch 200 Loss 0.7914
Epoch 2 Batch 300 Loss 0.7636
Epoch 2 Batch 400 Loss 0.8178
Epoch 2 Batch 500 Loss 0.8123
Epoch 2 Batch 600 Loss 0.6648
Epoch 2 Batch 700 Loss 0.6588
Epoch 2 Batch 800 Loss 0.7486
Epoch 2 Batch 900 Loss 0.7269
Epoch 2 Batch 1000 Loss 0.6116
Epoch 2 Batch 1100 Loss 0.5634
Epoch 2 Batch 1200 Loss 0.6617
Epoch 2 Loss 0.7133
Time taken for 1 epoch 1317.9228885173798 sec

Epoch 3 Batch 0 Loss 0.4588
Epoch 3 Batch 100 Loss 0.4522
Epoch 3 Batch 200 Loss 0.41

Epoch 18 Batch 1000 Loss 0.1072
Epoch 18 Batch 1100 Loss 0.0418
Epoch 18 Batch 1200 Loss 0.0793
Epoch 18 Loss 0.0680
Time taken for 1 epoch 1566.7207083702087 sec

Epoch 19 Batch 0 Loss 0.0518
Epoch 19 Batch 100 Loss 0.0379
Epoch 19 Batch 200 Loss 0.0580
Epoch 19 Batch 300 Loss 0.0827
Epoch 19 Batch 400 Loss 0.0661
Epoch 19 Batch 500 Loss 0.0660
Epoch 19 Batch 600 Loss 0.0687
Epoch 19 Batch 700 Loss 0.0714
Epoch 19 Batch 800 Loss 0.0563
Epoch 19 Batch 900 Loss 0.0627
Epoch 19 Batch 1000 Loss 0.0650
Epoch 19 Batch 1100 Loss 0.0704
Epoch 19 Batch 1200 Loss 0.1471
Epoch 19 Loss 0.0671
Time taken for 1 epoch 1522.8802633285522 sec

Epoch 20 Batch 0 Loss 0.0589
Epoch 20 Batch 100 Loss 0.0776
Epoch 20 Batch 200 Loss 0.0482
Epoch 20 Batch 300 Loss 0.0523
Epoch 20 Batch 400 Loss 0.0680
Epoch 20 Batch 500 Loss 0.0474
Epoch 20 Batch 600 Loss 0.0565
Epoch 20 Batch 700 Loss 0.0748
Epoch 20 Batch 800 Loss 0.1100
Epoch 20 Batch 900 Loss 0.0312
Epoch 20 Batch 1000 Loss 0.0750
Epoch 20 Batch 1100 Loss

In [31]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

    # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [32]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [33]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [34]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


In [35]:
translate(u'Я никогда такого не делаю.')

Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 
